# 문제 개요

이 문제는 실제 Programmers 채용 공고 페이지를 방문한 개발자들의 방문/지원 기록을 바탕으로 추천모델을 만들어보는 것이다. 참고로 모든 데이터는 익명화되었음을 밝힌다. 구체적으로 개발자(userID로 표현됨)와 채용공고(jobID로 표현됨)를 보고 개발자가 해당 채용공고에 지원할지 안 할지를 예측하는 Binary Classifier를 만드는 것이다. 이 모델을 만드는데 필요한 feature들은 부가적으로 제공되는 데이터 파일들을 바탕으로 만들어내면 되며 이에 대해서는 뒤의 데이터 세트 섹션에서 설명한다.

# 데이터 세트 설명

이 문제를 해결하는데 필요한 데이터들은 다음과 같은 6개의 파일에 들어있으며 각 파일에 관해 설명한다.

## train.csv
이 파일의 처음 5줄은 다음과 같다:

>userID,jobID,applied
>
>fe292163d06253b716e9a0099b4,0
>
>6377fa90618fae77571e8dc90d9,0
>
>8ec0888a5b04139be0dfe942c7e,1
>
>f862b39f767d3a1991bdeb2ea14,0

첫 번째 칼럼은 개발자의 ID이고 두 번째 칼럼은 구인공고(이후 “잡”이라 표현)의 ID가 된다. 마지막 세 번째 칼럼은 개발자가 해당 구인공고를 보고 지원을 했는지 여부를 나타낸다. 앞서 이야기했듯이 이번에 만들어볼 추천모델은 개발자와 잡의 페어를 보고 지원 여부를 예측하는 것이다. 이를 위해서는 개발자와 잡에 대한 세부 정보가 필요한데 뒤에 나오는 4개의 파일에서 그런 정보들(Features)을 추출할 수 있다.

## job_tags.csv
이 파일의 처음 5줄은 다음과 같다:

> jobID,tagID
>
>320722549d1751cf3f247855f937b982,d38901788c533e8286cb6400b40b386d
>
>e744f91c29ec99f0e662c9177946c627,3948ead63a9f2944218de038d8934305
>
>e744f91c29ec99f0e662c9177946c627,0e095e054ee94774d6a496099eb1cf6a
>
>e820a45f1dfc7b95282d10b6087e11c0,7d771e0e8f3633ab54856925ecdefc5d

이 파일은 각 잡의 키워드(tagID)들을 나타낸다. 하나의 잡에는 여러 개의 키워드가 존재할 수 있다. tagID에 해당하는 키워드가 뭔지 알고 싶다면 뒤에서 설명하는 tags.csv를 참조하면 된다.

## user_tags.csv
이 파일의 처음 5줄은 다음과 같다:

>userID,tagID
>
>e576423831e043f7928d9ac113abbe6f,82c2559140b95ccda9c6ca4a8b981f1e
>
>e576423831e043f7928d9ac113abbe6f,2ba8698b79439589fdd2b0f7218d8b07
>
>e576423831e043f7928d9ac113abbe6f,351b33587c5fdd93bd42ef7ac9995a28
>
>e576423831e043f7928d9ac113abbe6f,81e5f81db77c596492e6f1a5a792ed53

이 파일은 각 개발자가 관심사로 등록한 키워드(tagID)들을 나타낸다. 한 개발자가 여러 개의 키워드를 등록할 수 있다. 앞서 job_tags.csv처럼tagID에 해당하는 키워드가 뭔지 알고 싶다면 뒤에서 설명하는 tags.csv를 참조하면 된다. 예를 들어 tagID가 1189인 키워드는 JUnit이다.

## tags.csv
이 파일의 처음 5줄은 다음과 같다:

>tagID,keyword
>
>602d1305678a8d5fdb372271e980da6a,Amazon Web Services(AWS)
>
>e3251075554389fe91d17a794861d47b,Tensorflow
>
>a1d50185e7426cbb0acad1e6ca74b9aa,Docker
>
>884d79963bd8bc0ae9b13a1aa71add73,Git

앞에서 이 파일은 이미 두 번 설명한 셈인데 tagID에 해당하는 실제 키워드가 무엇인지 기술한다.

## job_companies.csv
이 파일의 처음 5줄은 다음과 같다:

>companyID,jobID,companySize
>
>00411460f7c92d2124a67ea0f4cb5f85,e5f6ad6ce374177eef023bf5d0c018b6,
>
>1905aedab9bf2477edc068a355bba31a,185e65bc40581880c4f2c82958de8cfe,11-50
>
>89fcd07f20b6785b92134bd6c1d0fa42,0537fb40a68c18da59a35c2bfe1ca554,101-200
>
>acf4b89d3d503d8252c9c4ba75ddbf6d,b7ee6f5f9aa5cd17ca1aea43ce848496,1-10

이 파일은 잡과 관련된 회사가 어디인지 그 정보를 보여주는데 별다른 정보는 없고 회사의 규모를 보여주는 필드가 존재하는데 이 필드는 옵션 필드이다 (값이 존재하지 않을 수 있음). 이 정보를 통해 어떤 잡들이 어느 회사에서 나온 것이고 그 회사의 크기는 어떻게 되는지 알 수 있다.

## test.csv
이 파일이 바로 최종 예측 모델의 입력이 되는 파일로 처음 다섯 줄은 다음과 같다:

>userID,jobID
>
>ebaee1af0c501f22ddfe242fc16dae53,352407221afb776e3143e8a1a0577885
>
>9ab05403ac7808cbfba3da26665f7a9c,96b9bff013acedfb1d140579e2fbeb63
>
>33349e909eba71677299d2fc97e158b7,58d4d1e7b1e97b258c9ed0b37e02d087
>
>ac985a9db5faeb44c94a334430ccc241,ccb0989662211f61edae2e26d58ea92f

라인마다 userID와 jobID가 있고 이 개발자가 이 잡에 지원을 할지 말지 예측을 하면 된다.

# 최종 제출 파일

- 최종 제출 파일은 applied 필드를 가지는 csv파일이며 test.csv에서 나오는 user가 지원했을지 여부를 값으로 가져야 한다. (1이면 지원을 나타내고 0이면 지원하지 않음을 나타냄)

- 최종 제출 파일의 n번째 줄값은 test.csv의 n번째 줄 사용자에 대한 예측값이어야 한다.


# 데이터 관련 안내

- 작성한 코드의 저작권은 응시자에게 있음

- 데이터의 저작권은 그렙에 있으므로 제3자에게 공유하거나 개인적인 용도로 사용할 수 없으며, 외부 블로그 등에 공개 불가

- 그렙에서 사용하기 위해 데이터를 제공하는 것이 아니며, 실전 데이터를 제공해서 평가 진행하기 위해 제공함

- 모델 학습을 위해 외부 데이터 및 모델(pre-trained model)을 사용 불가


# 채점 방식
- Accuracy 100%